In [3]:
import pandas as pd
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)


In [4]:
def get_numpy_data(data_dframe, features, output):
    data_dframe['constant'] = 1
    features = ['constant'] + features
    features_dframe = data_dframe[features]
    feature_matrix = features_dframe.to_numpy()
    output_darray = data_dframe[output] 
    output_array = output_darray.to_numpy()
    return(feature_matrix, output_array)

In [5]:
def predict_output(feature_matrix, weights):
    predicted_value = np.dot(feature_matrix, weights)
    return (predicted_value)

In [6]:
all_features = ['bedrooms',
            'bathrooms',
            'sqft_living',
            'sqft_lot',
            'floors',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [7]:
(example_features, example_output) = get_numpy_data(sales, all_features, 'price') 

### Sample 

In [8]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


In [9]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


In [10]:
print (X / norms)

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


In [11]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features/norms
    
    return (normalized_features, norms)
    

In [12]:
features, norms = normalize_features(example_features)

### SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)

### Sample

In [13]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [14]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [15]:
weights = np.array([1., 4., 1.])

In [16]:
prediction =  predict_output(simple_feature_matrix,weights)

In [17]:
for i in range(0,len(simple_features)):
    feature_i = simple_feature_matrix[:,i]
    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
    if i == 0:
        w[i] = ro[i]
    else:
        w[i]= ro[i] 

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])

In [108]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
      simple_feature_matrix, norms = normalize_features(feature_matrix)
      prediction =  predict_output(simple_feature_matrix,weights)
      feature_i = simple_feature_matrix[:,i]
      ro_i = sum(feature_i*(output - prediction + weights[i]*feature_i))
      if i == 0: 
        new_weight_i = ro_i 
      elif ro_i < -l1_penalty/2:
        new_weight_i = ro_i + l1_penalty/2
      elif ro_i > l1_penalty/2:
        new_weight_i = ro_i - l1_penalty/2
      else:
        new_weight_i = 0
    
      return new_weight_i

In [58]:
# should print 0.425558846691
import math
output = lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1)

## Cyclical coordinate descent

In [119]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    change_weight = []
    for i in range(len(weights)):
        old_weights_i = weights[i]
        weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        change_weight.append(abs(weights[i]))
    max_change = max(change_weight)
    if max_change < tolerance:
        return (max_change)
    else:
        print(weights)
        lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance)

In [70]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [97]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [118]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

[21624993.55123778 63157251.24051948        0.        ]


In [115]:
weights

63157251.24051948